In [2]:
!pip install pandas sklearn

  Using cached https://files.pythonhosted.org/packages/73/9b/52e228545d14f14bb2a1622e225f38463c8726645165e1cb7dde95bfe6d4/pandas-0.25.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/a0/c5/d2238762d780dde84a20b8c761f563fe882b88c5a5fb03c056547c442a19/scikit_learn-0.21.3-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/cd/c1/50a758e8247561e58cb87305b1e90b171b8c767b15b12a1734001f41d356/joblib-0.13.2-py2.py3-none-any.whl
You are using pip version 19.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import os
import numpy as np
import pandas as pd
from datetime import date

from sklearn.model_selection import KFold, train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

In [4]:
df_train = pd.read_csv('./T-BrainAI_dataset/train.csv', encoding='big5')
df_train_result = df_train[['CUS_ID', 'Y1']]
df_train_result['result'] = df_train_result['Y1'].apply(lambda x: 1 if x == 'Y' else 0)
df_train_result = df_train_result.drop(['Y1'], axis=1)
df_train = df_train[df_train.columns.difference(['Y1'])]
df_train

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (17,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,ACCIDENT_HOSPITAL_REC_AMT,AGE,AG_CNT,AG_NOW_CNT,ANNUAL_INCOME_AMT,ANNUAL_PREMIUM_AMT,ANNUITY_AMT,APC_1ST_AGE,APC_1ST_YEARDIF,APC_CNT,...,TERMINATION_RATE,TOOL_VISIT_1YEAR_CNT,X_A_IND,X_B_IND,X_C_IND,X_D_IND,X_E_IND,X_F_IND,X_G_IND,X_H_IND
0,NaN,低,1,0,0.001667,0.000192,NaN,低,0.046512,0,...,0.0,0,N,N,N,N,N,N,N,N
1,NaN,低,1,0,0.000167,0.009675,NaN,低,0.046512,0,...,0.0,0,N,N,N,N,N,N,N,N
2,0.0,低,1,0,0.000125,0.000292,0.0,低,0.000000,1,...,0.0,0,N,N,N,Y,N,N,N,N
3,NaN,低,1,0,0.004167,NaN,NaN,低,0.162791,0,...,0.0,0,N,N,N,N,N,N,N,N
4,0.0,低,1,1,0.000083,0.000664,0.0,低,0.116279,1,...,0.0,0,N,Y,N,N,Y,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,NaN,高,2,0,NaN,0.000483,NaN,中高,0.372093,0,...,0.0,1,N,N,Y,N,Y,N,N,N
99996,NaN,高,1,0,NaN,NaN,NaN,中高,0.465116,0,...,100.0,0,N,N,N,N,N,N,N,N
99997,NaN,中高,3,1,0.000625,NaN,NaN,中,0.279070,0,...,50.0,0,N,N,N,N,N,N,N,N
99998,NaN,高,1,1,0.002083,0.000548,NaN,高,0.023256,0,...,0.0,9,N,Y,Y,N,Y,N,N,Y


In [5]:
df_train_result

,CUS_ID,result
0,3418,0
1,4302,0
2,5545,0
3,7207,0
4,7213,0
...,...,...
99995,2525675,0
99996,2534462,1
99997,2535679,1
99998,2544819,0


In [6]:
df_test = pd.read_csv('./T-BrainAI_dataset/test.csv', encoding='big5')
df_test

,CUS_ID,GENDER,AGE,CHARGE_CITY_CD,CONTACT_CITY_CD,EDUCATION_CD,MARRIAGE_CD,LAST_A_CCONTACT_DT,L1YR_A_ISSUE_CNT,LAST_A_ISSUE_DT,...,IF_ADD_INSD_IND,L1YR_GROSS_PRE_AMT,CUST_9_SEGMENTS_CD,FINANCETOOLS_A,FINANCETOOLS_B,FINANCETOOLS_C,FINANCETOOLS_D,FINANCETOOLS_E,FINANCETOOLS_F,FINANCETOOLS_G
0,1193,M,低,A1,A1,2.0,0.0,Y,4,Y,...,Y,0.000151,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1727,M,低,A1,A1,NaN,0.0,N,0,N,...,N,0.000000,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3293,M,低,A1,A1,1.0,0.0,N,0,N,...,N,0.000090,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4328,M,低,A1,A1,NaN,0.0,N,0,N,...,N,0.000000,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5555,M,低,A1,A1,1.0,0.0,Y,0,Y,...,N,0.000179,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,2074085,F,中高,A1,A1,3.0,0.0,Y,0,N,...,NaN,0.000122,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149996,2095927,F,高,B1,B1,3.0,1.0,Y,1,Y,...,NaN,0.000104,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149997,2125973,M,高,C1,C1,3.0,1.0,Y,0,N,...,NaN,0.000225,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149998,2511714,F,高,D,D,3.0,1.0,Y,0,N,...,NaN,0.000000,H,Y,N,N,N,N,N,N


In [7]:
nonNum_features = []
for feature in df_train.columns:
    try:
        if df_train[feature].dtype == object:
            nonNum_features.append(feature)
    except TypeError as e:
        print(e)

print(f'{len(nonNum_features)} Non-num Features : {nonNum_features}\n')

Num_features = []
for feature in df_train.columns:
    try:
        if (df_train[feature].dtype == int) or (df_train[feature].dtype == float):
            Num_features.append(feature)
    except TypeError as e:
        print(e)

print(f'{len(Num_features)} Num Features : {Num_features}\n')

89 Non-num Features : ['AGE', 'APC_1ST_AGE', 'A_IND', 'B_IND', 'CHARGE_CITY_CD', 'CONTACT_CITY_CD', 'CUST_9_SEGMENTS_CD', 'C_IND', 'FINANCETOOLS_A', 'FINANCETOOLS_B', 'FINANCETOOLS_C', 'FINANCETOOLS_D', 'FINANCETOOLS_E', 'FINANCETOOLS_F', 'FINANCETOOLS_G', 'GENDER', 'IF_2ND_GEN_IND', 'IF_ADD_F_IND', 'IF_ADD_G_IND', 'IF_ADD_IND', 'IF_ADD_INSD_F_IND', 'IF_ADD_INSD_G_IND', 'IF_ADD_INSD_IND', 'IF_ADD_INSD_L_IND', 'IF_ADD_INSD_Q_IND', 'IF_ADD_INSD_R_IND', 'IF_ADD_L_IND', 'IF_ADD_Q_IND', 'IF_ADD_R_IND', 'IF_HOUSEHOLD_CLAIM_IND', 'IF_ISSUE_A_IND', 'IF_ISSUE_B_IND', 'IF_ISSUE_C_IND', 'IF_ISSUE_D_IND', 'IF_ISSUE_E_IND', 'IF_ISSUE_F_IND', 'IF_ISSUE_G_IND', 'IF_ISSUE_H_IND', 'IF_ISSUE_INSD_A_IND', 'IF_ISSUE_INSD_B_IND', 'IF_ISSUE_INSD_C_IND', 'IF_ISSUE_INSD_D_IND', 'IF_ISSUE_INSD_E_IND', 'IF_ISSUE_INSD_F_IND', 'IF_ISSUE_INSD_G_IND', 'IF_ISSUE_INSD_H_IND', 'IF_ISSUE_INSD_I_IND', 'IF_ISSUE_INSD_J_IND', 'IF_ISSUE_INSD_K_IND', 'IF_ISSUE_INSD_L_IND', 'IF_ISSUE_INSD_M_IND', 'IF_ISSUE_INSD_N_IND', 'IF_I

In [8]:
selected_features = ['CUS_ID', 'L1YR_A_ISSUE_CNT', 'L1YR_A_ISSUE_CNT', 'IF_ISSUE_I_IND', 'LAST_A_CCONTACT_DT', 'AG_NOW_CNT', 
                     'IF_ISSUE_Q_IND', 'CHANNEL_A_POL_CNT', 'TOOL_VISIT_1YEAR_CNT', 'IF_ADD_Q_IND', 'IF_ADD_L_IND', 'LEVEL', 'IF_ADD_IND', 
                     'CLC_CUR_NUM', 'BANK_NUMBER_CNT', 'IF_ISSUE_J_IND', 'LIFE_INSD_CNT', 'LAST_C_DT', 'IM_CNT', 'EDUCATION_CD',
                     'INSD_LAST_YEARDIF_CNT', 'IF_2ND_GEN_IND', 'APC_CNT', 'TERMINATION_RATE', 'APC_1ST_YEARDIF', 'AGE']

df_train = df_train[selected_features]
df_test = df_test[selected_features]

fill_values = {
    'INSD_LAST_YEARDIF_CNT': 0.1724875816240972,
    'EDUCATION_CD': 3.0, 
    'APC_1ST_YEARDIF': 0.2587685984593603, 
    'LEVEL': 5.0, 
    'TERMINATION_RATE': 12.090984696216369
}

for i,j in fill_values.items():
    df_train[i] = df_train[i].fillna(value=fill_values[i])
    df_test[i] = df_test[i].fillna(value=fill_values[i])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:
# one hot encoding - get_dummies
# df_train_num = df_train[Num_features].fillna('None')
# df_train_nonNum = df_train[nonNum_features].fillna('None')

t = pd.get_dummies(df_train)
df_train = pd.DataFrame(t)
tt = pd.get_dummies(df_test)
df_test = pd.DataFrame(tt)
df_train

,CUS_ID,L1YR_A_ISSUE_CNT,L1YR_A_ISSUE_CNT,AG_NOW_CNT,CHANNEL_A_POL_CNT,TOOL_VISIT_1YEAR_CNT,LEVEL,CLC_CUR_NUM,BANK_NUMBER_CNT,LIFE_INSD_CNT,...,IF_ISSUE_J_IND_N,IF_ISSUE_J_IND_Y,LAST_C_DT_N,LAST_C_DT_Y,IF_2ND_GEN_IND_N,IF_2ND_GEN_IND_Y,AGE_中,AGE_中高,AGE_低,AGE_高
0,3418,0,0,0,0,0,3.0,0,0.125,0.015625,...,1,0,1,0,1,0,0,0,1,0
1,4302,0,0,0,0,0,5.0,0,0.125,0.015625,...,1,0,1,0,1,0,0,0,1,0
2,5545,1,1,0,1,0,5.0,1,0.000,0.031250,...,1,0,1,0,0,1,0,0,1,0
3,7207,0,0,0,0,0,5.0,0,0.125,0.015625,...,1,0,1,0,1,0,0,0,1,0
4,7213,0,0,1,1,0,2.0,1,0.000,0.062500,...,1,0,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2525675,0,0,0,3,1,1.0,2,0.000,0.000000,...,1,0,1,0,1,0,0,0,0,1
99996,2534462,0,0,0,1,0,5.0,0,0.000,0.000000,...,1,0,1,0,1,0,0,0,0,1
99997,2535679,0,0,1,4,0,5.0,0,0.000,0.000000,...,1,0,1,0,1,0,0,1,0,0
99998,2544819,0,0,1,2,9,4.0,1,0.125,0.000000,...,1,0,1,0,1,0,0,0,0,1


In [10]:
# split train and valid
train_x = df_train[:50000]
valid_x = df_train[50000:]
train_y = df_train_result[:50000]
valid_y = df_train_result[50000:]

In [11]:
r = df_train.drop('CUS_ID',axis=1)
r = r.columns
r

rr = df_test.drop('CUS_ID',axis=1)
rr = rr.columns
rr

Index(['L1YR_A_ISSUE_CNT', 'L1YR_A_ISSUE_CNT', 'AG_NOW_CNT',
       'CHANNEL_A_POL_CNT', 'TOOL_VISIT_1YEAR_CNT', 'LEVEL', 'CLC_CUR_NUM',
       'BANK_NUMBER_CNT', 'LIFE_INSD_CNT', 'IM_CNT', 'EDUCATION_CD',
       'INSD_LAST_YEARDIF_CNT', 'APC_CNT', 'TERMINATION_RATE',
       'APC_1ST_YEARDIF', 'IF_ISSUE_I_IND_N', 'IF_ISSUE_I_IND_Y',
       'LAST_A_CCONTACT_DT_N', 'LAST_A_CCONTACT_DT_Y', 'IF_ISSUE_Q_IND_N',
       'IF_ISSUE_Q_IND_Y', 'IF_ADD_Q_IND_N', 'IF_ADD_Q_IND_Y',
       'IF_ADD_L_IND_N', 'IF_ADD_L_IND_Y', 'IF_ADD_IND_N', 'IF_ADD_IND_Y',
       'IF_ISSUE_J_IND_N', 'IF_ISSUE_J_IND_Y', 'LAST_C_DT_N', 'LAST_C_DT_Y',
       'IF_2ND_GEN_IND_N', 'IF_2ND_GEN_IND_Y', 'AGE_中', 'AGE_中高', 'AGE_低',
       'AGE_高'],
      dtype='object')

In [12]:
def check_model(data, r, df_train_result):
    
    classifier = lambda: SGDClassifier(
        loss='log', 
        penalty='elasticnet', 
        fit_intercept=True, 
        max_iter=100, 
        shuffle=True, 
#         n_jobs=-1,
        class_weight=None)

    model = Pipeline(steps=[
        ('ss', StandardScaler()),
        ('en', classifier())
    ])

    parameters = {
        'en__alpha': [ 0.001, 0.01, 0.1],
        'en__l1_ratio': [ 0.001, 0.01, 0.1]
    }

    folder = StratifiedKFold(n_splits=3, shuffle=True)
    
    grid_search = GridSearchCV(
        model, 
        parameters, 
        cv=folder, 
        n_jobs=-1, 
        verbose=1)
    grid_search = grid_search.fit(data[r], df_train_result['result'])
    
    return grid_search

In [13]:
model = check_model(train_x, r, train_y)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 out of  27 | elapsed:    3.9s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    4.2s finished


In [14]:
valid_y_pred = model.predict_proba(valid_x[r])
valid_x_1 = valid_x.copy()
valid_x_1['predict_proba'] = valid_y_pred[:, 1]

In [15]:
auc_score = roc_auc_score(y_true=valid_y.result, y_score=valid_y_pred[:, 1])
acc = accuracy_score(y_true=valid_y.result, y_pred=valid_y_pred.argmax(axis=1))
print('Validation AUC: {:.3f}, Accuracy: {:.3f}'.format(auc_score, acc))

Validation AUC: 0.646, Accuracy: 0.978


In [ ]:
# training process

from matplotlib import pyplot as plt
plt.plot(valid_y_pred['acc'])
plt.plot(history.history['val_acc'])
plt.title('acc-')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')
plt.savefig('history')
plt.show()

In [23]:
targetset = df_test.copy()
print(targetset.shape)

test_y_pred = model.predict_proba(targetset[r])
test_x_1 = targetset.copy()
test_x_1['predict_proba'] = test_y_pred[:, 1]
print(test_x_1.shape)

(150000, 38)
(150000, 39)


In [24]:
test_y_pred

array([[0.99703569, 0.00296431],
       [0.99833942, 0.00166058],
       [0.99781315, 0.00218685],
       ...,
       [0.99675127, 0.00324873],
       [0.99679327, 0.00320673],
       [0.99807962, 0.00192038]])

In [25]:
output = pd.concat((targetset[['CUS_ID']], test_x_1['predict_proba']), axis=1)
print(output.shape)
output

(150000, 2)


,CUS_ID,predict_proba
0,1193,0.002964
1,1727,0.001661
2,3293,0.002187
3,4328,0.001661
4,5555,0.002854
...,...,...
149995,2074085,0.003136
149996,2095927,0.003188
149997,2125973,0.003249
149998,2511714,0.003207


In [26]:
out = output.groupby('CUS_ID', as_index=False).mean()
out = out[['CUS_ID', 'predict_proba']]
out.columns = ['CUS_ID', 'Ypred']
out.to_csv('output_190922_1_sklearn_SGDC_GridSearchCV_allFeature.csv')
out

,CUS_ID,Ypred
0,3,0.001563
1,5,0.001726
2,20,0.001725
3,48,0.001725
4,49,0.001745
...,...,...
149995,2591999,0.001761
149996,2592019,0.002480
149997,2592023,0.004268
149998,2592034,0.001822
